
#Setting Up the Models



##Install Libraries and Import Packages

In [ ]:
!pip install -r requirements_rg.txt

In [ ]:
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from interpret.glassbox import ExplainableBoostingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import  mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

from interpret import show
import shap
import graphviz
from sklearn.tree import export_graphviz

from scipy.stats import spearmanr
from collections import Counter

import random

# fetch dataset
parkinsons_telemonitoring = fetch_ucirepo(id=189)

# data (as pandas dataframes)
df = parkinsons_telemonitoring.data.original.copy()

SEED = 100
np.random.seed(SEED)
random.seed(SEED)

##Handling Outliers with IQR Method

In [ ]:
for col in df.columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df[col] = np.where(df[col] > upper, upper, df[col])
    df[col] = np.where(df[col] < lower, lower, df[col])

##Split Data

In [ ]:
X = df.drop(['total_UPDRS', 'motor_UPDRS', 'subject#', 'age'], axis=1)
y = df['total_UPDRS']

#change column names for LightGBM
X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

##Creating the Optimized Models and Then Fitting them

In [ ]:
dt = DecisionTreeRegressor(ccp_alpha=0.0, criterion='squared_error', max_depth=19,
                           max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                           min_samples_leaf=14, min_samples_split=5, min_weight_fraction_leaf=0.0,
                           monotonic_cst=None, random_state=100, splitter='random')

ebm = ExplainableBoostingRegressor(callback=None, cat_smooth=10.0, cyclic_progress=False,
                                   early_stopping_rounds=100, early_stopping_tolerance=1e-05, exclude=None,
                                   feature_names=None, feature_types=None, gain_scale=5.0,
                                   greedy_ratio=10.0, inner_bags=0, interaction_smoothing_rounds=100,
                                   interactions=6, learning_rate=0.0027763309052116383, max_bins=484,
                                   max_delta_step=0.0, max_interaction_bins=263, max_leaves=28,
                                   max_rounds=50000, min_cat_samples=10, min_hessian=0.0,
                                   min_samples_leaf=15, missing='separate', monotone_constraints=None,
                                   n_jobs=-2, objective='rmse', outer_bags=14,
                                   random_state=100, reg_alpha=0.0, reg_lambda=0.0,
                                   smoothing_rounds=500, validation_size=0.15)

cat = CatBoostRegressor(iterations=816, learning_rate=0.1261192471208588, depth=10,
                        l2_leaf_reg=9.06507208082491, loss_function='RMSE', bootstrap_type='MVS',
                        random_state=100)

lgbm = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7507121131532456,
                     importance_type='split', learning_rate=0.042853382162136154, max_depth=18,
                     min_child_samples=8, min_child_weight=0.001, min_split_gain=0.0,
                     n_estimators=768, n_jobs=None, num_leaves=355,
                     objective=None, random_state=100, reg_alpha=2.592134383897951,
                     reg_lambda=6.6855134447038855, subsample=0.9198022636755252, subsample_for_bin=200000,
                     subsample_freq=0)

In [ ]:
dt.fit(X_train, y_train)
ebm.fit(X_train, y_train)
cat.fit(X_train, y_train)
lgbm.fit(X_train, y_train)

##Evaluating Each Model's Performance

In [ ]:
#For dt
dt_pred = dt.predict(X_test)
print("\nFor the Decision Tree Model:")
print("\nMSE is : " + str(mean_squared_error(y_test, dt_pred)))
print("\nR2 is : " + str(r2_score(y_test, dt_pred)))
print("\nMAE is : " + str(mean_absolute_error(y_test, dt_pred)))

In [ ]:
#For ebm
ebm_pred = ebm.predict(X_test)
print("\nFor the Explainable Boosting Machine Model:")
print("\nMSE is : " + str(mean_squared_error(y_test, ebm_pred)))
print("\nR2 is : " + str(r2_score(y_test, ebm_pred)))
print("\nMAE is : " + str(mean_absolute_error(y_test, ebm_pred)))

In [ ]:
#For cat
cat_pred = cat.predict(X_test)
print("\nFor the CatBoost Model:")
print("\nMSE is : " + str(mean_squared_error(y_test, cat_pred)))
print("\nR2 is : " + str(r2_score(y_test, cat_pred)))
print("\nMAE is : " + str(mean_absolute_error(y_test, cat_pred)))

In [ ]:
#For lgbm
lgbm_pred = lgbm.predict(X_test)
print("\nFor the LightGBM Model:")
print("\nMSE is : " + str(mean_squared_error(y_test, lgbm_pred)))
print("\nR2 is : " + str(r2_score(y_test, lgbm_pred)))
print("\nMAE is : " + str(mean_absolute_error(y_test, lgbm_pred)))

#Global Explainability

##Decision Tree's Tree Structure

In [ ]:
dot_data = export_graphviz(dt, out_file=None,
                                feature_names=X.columns,
                                filled=True)

graph = graphviz.Source(dot_data, format="png")
graph

##EBM Global Explainability

In [ ]:
ebm_global = ebm.explain_global()
show(ebm_global)

##Setting Up SHAP explainers

In [ ]:
#turn data into DF for SHAP plots
X_test_df = pd.DataFrame(X_test, columns=X.columns)

#SHAP for dt
dt_explainer = shap.TreeExplainer(dt)
dt_shap_values = dt_explainer.shap_values(X_test_df)

#wrapper function for EBM predict to handle feature names
def ebm_predict_wrapper(X):
    X_df = pd.DataFrame(X, columns=X_train.columns)
    return ebm.predict(X_df)

#SHAP for ebm
ebm_explainer = shap.KernelExplainer(ebm_predict_wrapper, X_train, seed=SEED)
ebm_shap_values = ebm_explainer.shap_values(X_test_df)

#SHAP for cat
cat_explainer = shap.TreeExplainer(cat)
cat_shap_values = cat_explainer.shap_values(X_test_df)

#SHAP for lgbm
lgbm_explainer = shap.Explainer(lgbm)
lgbm_shap_values = lgbm_explainer.shap_values(X_test_df)

##SHAP Summary Plots For Each Model

In [ ]:
#dt
shap.summary_plot(dt_shap_values, X_test_df)

In [ ]:
#ebm
shap.summary_plot(ebm_shap_values, X_test_df)

In [ ]:
#cat
shap.summary_plot(cat_shap_values, X_test_df)

In [ ]:
#lgbm
shap.summary_plot(lgbm_shap_values, X_test_df)

#Local Explainability

##Selecting Instance

In [ ]:
index = 0

##Decision Path

In [ ]:
X_instance = X_test.iloc[[index]]

node_indicator = dt.decision_path(X_instance)
leaf_id = dt.apply(X_instance)

print(f"\nDecision path for instance {index}:")
for node_id in node_indicator.indices:
    if dt.tree_.children_left[node_id] != dt.tree_.children_right[node_id]:
        feature = X_test.columns[dt.tree_.feature[node_id]]
        threshold = dt.tree_.threshold[node_id]
        if X_instance.iloc[0, dt.tree_.feature[node_id]] <= threshold:
            threshold_sign = "<="
        else:
            threshold_sign = ">"
        print(f"  {feature} = {X_instance.iloc[0, dt.tree_.feature[node_id]]:.2f} "
              f"{threshold_sign} {threshold:.2f}")

pred_value = dt.predict(X_instance)[0]
true_value = y_test.iloc[index] if isinstance(y_test, pd.Series) else y_test[index]

print(f"\nPredicted value: {pred_value}")
print(f"Actual value: {true_value}")

##Local EBM Explainability

In [ ]:
ebm_local = ebm.explain_local(X_test, y_test)
show(ebm_local)

##SHAP Waterfalls

In [ ]:
#dt
shap.initjs()
shap.force_plot(dt_explainer.expected_value, dt_shap_values[index, :], X_test_df.iloc[index])

In [ ]:
#ebm
shap.initjs()
shap.force_plot(ebm_explainer.expected_value, ebm_shap_values[index, :], X_test_df.iloc[index])

In [ ]:
#cat
shap.initjs()
shap.force_plot(cat_explainer.expected_value, cat_shap_values[index, :], X_test_df.iloc[index])

In [ ]:
#lgbm
shap.initjs()
shap.force_plot(lgbm_explainer.expected_value, lgbm_shap_values[index, :], X_test_df.iloc[index])

#SHAP Evaluation

##Setting up Importances for EBM

In [ ]:
term_names = np.array(ebm.term_names_)
term_importances = np.array(ebm.term_importances())
print(term_names)

In [ ]:
main_mask = np.array([' & ' not in name for name in term_names])
main_features = term_names[main_mask]
main_importances = term_importances[main_mask]

In [ ]:
ebm_importances = np.zeros(len(X.columns))
for i, feature in enumerate(X.columns):
    if feature in main_features:
        idx = list(main_features).index(feature)
        ebm_importances[i] = main_importances[idx]

In [ ]:
models_dict = {
    'DT': (dt, dt_explainer, dt_shap_values, dt.feature_importances_),
    'EBM': (ebm, ebm_explainer, ebm_shap_values, ebm_importances),
    'CAT': (cat, cat_explainer, cat_shap_values, cat.feature_importances_),
    'LGBM': (lgbm, lgbm_explainer, lgbm_shap_values, lgbm.feature_importances_)
}

##Fidelity (Correlation between model and SHAP feature importances)

In [ ]:
def fidelity(model_importance, shap_values):
    shap_importance = np.abs(shap_values).mean(axis=0)
    return spearmanr(model_importance, shap_importance)[0]

##Consistency (Entropy of top feature across instances)

In [ ]:
def consistency(shap_values, feature_names):
    top_feature = np.argmax(np.abs(shap_values), axis=1)

    value, counts = np.unique(top_feature, return_counts=True)
    probs = counts / len(top_feature)
    entropy = -np.sum(probs * np.log(probs))

    dominant_feature = feature_names[value[np.argmax(counts)]]
    dominant_percent = counts.max() / len(top_feature)

    return entropy, dominant_feature, dominant_percent

##Robustness (Test if SHAP values remain stable under small perturbations)

In [ ]:
def robustness(explainer, X_sample_df, n_instances=10, n_perturbations=10, noise_std=0.1, seed=100):
    np.random.seed(seed)
    stabilities = []

    for i in range(min(n_instances, len(X_sample_df))):
        instance_df = X_sample_df.iloc[i:i+1]
        base_shap = explainer.shap_values(instance_df)

        corrs = []
        for _ in range(n_perturbations):
            noise = np.random.normal(0, noise_std, instance_df.shape)
            perturbed_df = pd.DataFrame(instance_df.values + noise, columns=instance_df.columns)
            perturbed_shap = explainer.shap_values(perturbed_df)
            corr = spearmanr(base_shap.flatten(), perturbed_shap.flatten())[0]
            corrs.append(corr)

        stabilities.append(np.mean(corrs))

    return np.mean(stabilities)

##Sufficiency (Test if top-k SHAP features preserve predictions)




In [ ]:
def sufficiency(model, X_test, shap_values, feature_names, k=5, n_samples=30):
    abs_errors = []
    rel_errors = []
    all_top_features = []

    for i in range(min(n_samples, len(X_test))):
        X_instance = X_test.iloc[i:i+1]
        original_pred = model.predict(X_instance)[0]

        shap_vals = shap_values[i, :]

        top_k_indices = np.argsort(np.abs(shap_vals))[-k:]
        all_top_features.extend(top_k_indices)

        masked_instance = pd.DataFrame(np.zeros((1, X_test.shape[1])), columns=X_test.columns)
        masked_instance.iloc[0, top_k_indices] = X_test.iloc[i, top_k_indices]

        masked_pred = model.predict(masked_instance)[0]

        abs_error = abs(original_pred - masked_pred)
        rel_error = abs_error / (abs(original_pred) + 1e-10)

        abs_errors.append(abs_error)
        rel_errors.append(rel_error)

    feature_counter = Counter(all_top_features)

    feature_usage = []
    for feat_idx in range(len(feature_names)):
        count = feature_counter.get(feat_idx, 0)
        percent = (count / n_samples) * 100
        feature_usage.append({'feature': feature_names[feat_idx], 'count': count, 'percentage': percent})

    feature_usage_df = pd.DataFrame(feature_usage).sort_values('count', ascending=False)

    top_features_dict = {
        feature_names[idx]: count
        for idx, count in feature_counter.most_common(k)
    }

    threshold = 0.10
    predictions_maintained = np.mean([err < threshold for err in rel_errors])

    return {
        'avg_abs_error': np.mean(abs_errors),
        'avg_rel_error': np.mean(rel_errors),
        'percent_maintained': predictions_maintained,
        'top_features_used': top_features_dict,
        'feature_usage_df': feature_usage_df
    }

##Completeness (Test if top-k SHAP features removals preserve predictions)

In [ ]:
def completeness(model, X_test, shap_values, feature_names, k=5, n_samples=30):
    abs_errors = []
    rel_errors = []
    all_removed_features = []

    for i in range(min(n_samples, len(X_test))):
        X_instance = X_test.iloc[i:i+1]
        original_pred = model.predict(X_instance)[0]

        shap_vals = shap_values[i, :]
        top_k_indices = np.argsort(np.abs(shap_vals))[-k:]
        all_removed_features.extend(top_k_indices)

        masked_instance = X_instance.copy()
        masked_instance.iloc[0, top_k_indices] = 0

        masked_pred = model.predict(masked_instance)[0]

        abs_error = abs(original_pred - masked_pred)
        rel_error = abs_error / (abs(original_pred) + 1e-10)

        abs_errors.append(abs_error)
        rel_errors.append(rel_error)

    feature_counter = Counter(all_removed_features)

    feature_usage = []
    for feat_idx in range(len(feature_names)):
        count = feature_counter.get(feat_idx, 0)
        percent = (count / n_samples) * 100
        feature_usage.append({'feature': feature_names[feat_idx], 'count': count, 'percentage': percent})

    feature_usage_df = pd.DataFrame(feature_usage).sort_values('count', ascending=False)

    top_features_dict = {feature_names[idx]: count for idx, count in feature_counter.most_common(k)}

    threshold = 0.10
    predictions_maintained = np.mean([err < threshold for err in rel_errors])

    return {
        'avg_abs_error': np.mean(abs_errors),
        'avg_rel_error': np.mean(rel_errors),
        'percent_maintained': predictions_maintained,
        'top_features_removed': top_features_dict,
        'feature_usage_df': feature_usage_df
    }

##SHAP Evaluation Results

In [ ]:
results = {}

for model_name, (model, explainer, shap_vals, feat_imp) in models_dict.items():
    fl_score = fidelity(feat_imp, shap_vals)

    cs_entropy, dominant_feature, dominant_percent = consistency(shap_vals, X.columns)
    cs_score = 1 - (cs_entropy / np.log(len(X.columns)))#min-max normalization

    rb_baseline = robustness(explainer, X_test_df.head(10), seed=SEED)
    rb_more_instances = robustness(explainer, X_test_df.head(30), n_instances=30, seed=SEED)
    rb_more_perturbations = robustness(explainer, X_test_df.head(10), n_perturbations=30, seed=SEED)
    rb_higher_noise = robustness(explainer, X_test_df.head(10), noise_std=0.3, seed=SEED)

    sufficiency_k1 = sufficiency(model, X_test, shap_vals, X.columns, k=1)
    sufficiency_k3 = sufficiency(model, X_test, shap_vals, X.columns, k=3)
    sufficiency_k5 = sufficiency(model, X_test, shap_vals, X.columns, k=5)
    sufficiency_k8 = sufficiency(model, X_test, shap_vals, X.columns, k=8)

    completeness_k1 = completeness(model, X_test, shap_vals, X.columns, k=1)
    completeness_k3 = completeness(model, X_test, shap_vals, X.columns, k=3)
    completeness_k5 = completeness(model, X_test, shap_vals, X.columns, k=5)
    completeness_k8 = completeness(model, X_test, shap_vals, X.columns, k=8)

    results[model_name] = {
        'fidelity': fl_score,
        'consistency': cs_score,
        'dominant_feature': dominant_feature,
        'dominant_percent': dominant_percent,
        'robustness_baseline': rb_baseline,
        'robustness_more_instances': rb_more_instances,
        'robustness_more_perturbations': rb_more_perturbations,
        'robustness_higher_noise': rb_higher_noise,
        'sufficiency_k1': sufficiency_k1,
        'sufficiency_k3': sufficiency_k3,
        'sufficiency_k5': sufficiency_k5,
        'sufficiency_k8': sufficiency_k8,
        'completeness_k1': completeness_k1,
        'completeness_k3': completeness_k3,
        'completeness_k5': completeness_k5,
        'completeness_k8': completeness_k8
    }

##Results per Model

In [ ]:
model_names = ['DT', 'EBM', 'CAT', 'LGBM']

for model in model_names:
    print(f"\nModel: {model}")
    for metric in ['fidelity', 'consistency', 'robustness_baseline', 'dominant_feature', 'dominant_percent']:
        print(f"{metric}: {results[model][metric]}")

##Fidelity Plot

In [ ]:
fl_scores = [results[m]['fidelity'] for m in model_names]
plt.bar(model_names, fl_scores)
plt.ylabel('Correlation')
plt.title('Fidelity')
plt.show()

##Consistency Plot

In [ ]:
cs_scores = [results[m]['consistency'] for m in model_names]
plt.bar(model_names, cs_scores)
plt.ylabel('Correlation')
plt.title('Consistency')
plt.show()

##Robustness Testing Results for each Model

In [ ]:
for model in model_names:
    print(f"\nModel: {model}")
    for metric in ['robustness_baseline', 'robustness_more_instances', 'robustness_more_perturbations', 'robustness_higher_noise']:
        print(f"{metric}: {results[model][metric]}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(model_names))
width = 0.2

rects1 = ax.bar(x - 1.5*width, [results[m]['robustness_baseline'] for m in model_names], width, label='RB 1')
rects2 = ax.bar(x - 0.5*width, [results[m]['robustness_more_instances'] for m in model_names], width, label='RB 2')
rects3 = ax.bar(x + 0.5*width, [results[m]['robustness_more_perturbations'] for m in model_names], width, label='RB 3')
rects4 = ax.bar(x + 1.5*width, [results[m]['robustness_higher_noise'] for m in model_names], width, label='RB 4')

ax.set_ylabel('Correlation')
ax.set_title('Robustness by Model')
ax.set_xticks(x)
ax.set_xticklabels(model_names)
ax.legend(['RB 1 (n_instances=10)', 'RB 2 (n_instances=30)', 'RB 3 (n_perturbations=30)', 'RB 4 (noise_std=0.3)'])

plt.tight_layout()
plt.show()

##Suffiency Testing Results for each Model

In [ ]:
dt_top5 = set(results['DT']['sufficiency_k5']['feature_usage_df'].head(5)['feature'])
ebm_top5 = set(results['EBM']['sufficiency_k5']['feature_usage_df'].head(5)['feature'])
cat_top5 = set(results['CAT']['sufficiency_k5']['feature_usage_df'].head(5)['feature'])
lgbm_top5 = set(results['LGBM']['sufficiency_k5']['feature_usage_df'].head(5)['feature'])

print(f"Decision Tree Top 5: {dt_top5}")
print(f"EBM Top 5: {ebm_top5}")
print(f"CatBoost Top 5: {cat_top5}")
print(f"LightGBM Top 5: {lgbm_top5}")

In [ ]:
percent_maintained_k1 = [results[m]['sufficiency_k1']['percent_maintained'] for m in model_names]
percent_maintained_k3 = [results[m]['sufficiency_k3']['percent_maintained'] for m in model_names]
percent_maintained_k5 = [results[m]['sufficiency_k5']['percent_maintained'] for m in model_names]
percent_maintained_k8 = [results[m]['sufficiency_k8']['percent_maintained'] for m in model_names]

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(model_names))
width = 0.2

rects1 = ax.bar(x - 1.5*width, percent_maintained_k1, width, label='k=1', alpha=0.8)
rects2 = ax.bar(x - 0.5*width, percent_maintained_k3, width, label='k=3', alpha=0.8)
rects3 = ax.bar(x + 0.5*width, percent_maintained_k5, width, label='k=5', alpha=0.8)
rects4 = ax.bar(x + 1.5*width, percent_maintained_k8, width, label='k=8', alpha=0.8)

ax.set_ylabel('% Predictions Maintained (<10% error)', fontsize=11)
ax.set_title('Sufficiency Test: Prediction Preservation for Different k', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names)
ax.legend()
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

##Completeness Testing Results for each Model

In [ ]:
percent_maintained_k1 = [results[m]['completeness_k1']['percent_maintained'] for m in model_names]
percent_maintained_k3 = [results[m]['completeness_k3']['percent_maintained'] for m in model_names]
percent_maintained_k5 = [results[m]['completeness_k5']['percent_maintained'] for m in model_names]
percent_maintained_k8 = [results[m]['completeness_k8']['percent_maintained'] for m in model_names]

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(model_names))
width = 0.2

rects1 = ax.bar(x - 1.5*width, percent_maintained_k1, width, label='k=1', alpha=0.8)
rects2 = ax.bar(x - 0.5*width, percent_maintained_k3, width, label='k=3', alpha=0.8)
rects3 = ax.bar(x + 0.5*width, percent_maintained_k5, width, label='k=5', alpha=0.8)
rects4 = ax.bar(x + 1.5*width, percent_maintained_k8, width, label='k=8', alpha=0.8)

ax.set_ylabel('% Predictions Maintained (<10% error)', fontsize=11)
ax.set_title('Completeness Test: Prediction Preservation for Different k', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names)
ax.legend()
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

##R2 Score vs Explainability Plot (Through mean values of metrics)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

explain_scores = {
    m: np.mean([
        results[m]['fidelity'],
        results[m]['robustness_baseline'],
        results[m]['consistency'],
        results[m]['sufficiency_k5']['percent_maintained'],
        results[m]['completeness_k3']['percent_maintained']
    ]) for m in model_names
}

r2_scores = {
    'DT': r2_score(y_test, dt_pred),
    'EBM': r2_score(y_test, ebm_pred),
    'CAT': r2_score(y_test, cat_pred),
    'LGBM': r2_score(y_test, lgbm_pred)
}

for model in model_names:
    ax.scatter(explain_scores[model], r2_scores[model])
    ax.annotate(model, (explain_scores[model], r2_scores[model]))

ax.set_xlabel('Explainability Score (0-1)')
ax.set_ylabel('Model R2 Score')
ax.set_title('R2 Score vs Explainability Tradeoff')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
for model in model_names:
    print(f"{model}, {explain_scores[model]}, {r2_scores[model]}")